In [9]:
import json
import jiwer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge

In [10]:
# Función auxiliar para asegurarse de que siempre se obtiene una lista de tokens
def safe_split(x):
    if isinstance(x, str):
        tokens = x.split()
        return tokens if tokens else ["<EMPTY>"]
    return x

# Función para calcular el Token Error Rate (Word Error Rate)
def compute_token_error_rate(reference, hypothesis):
    # Componer la transformación para normalizar ambos textos
    transformation = jiwer.Compose([
        jiwer.ExpandCommonEnglishContractions(),
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords()
    ])
    # Calcula el Word Error Rate
    error_rate = jiwer.wer(
        reference, 
        hypothesis, 
        truth_transform=transformation, 
        hypothesis_transform=transformation
    )
    return error_rate

# Función para calcular el BLEU
def compute_bleu(reference, hypothesis):
    ref_tokens = reference.split()
    hyp_tokens = hypothesis.split()
    smoothie = SmoothingFunction().method4
    bleu_score = sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=smoothie)
    return bleu_score

# Función para calcular ROUGE-L
def compute_rouge(reference, hypothesis):
    rouge_evaluator = Rouge()
    scores = rouge_evaluator.get_scores(hypothesis, reference)
    rouge_l = scores[0]['rouge-l']['f']
    return rouge_l

# Función para calcular las métricas para cada respuesta
def calculate_metrics(results):
    for result in results:
        expected = result["expected_answer"].strip()
        generated = result["generated_answer"].strip()
        
        result["token_error_rate"] = compute_token_error_rate(expected, generated)
        result["bleu_score"] = compute_bleu(expected, generated)
        try:
            result["rouge_l"] = compute_rouge(expected, generated)
        except Exception as e:
            result["rouge_l"] = None
            print(f"Error al calcular ROUGE: {e}")
    return results

In [11]:
# Abrir los archivos JSON con los resultados
with open("non_expert_results.json", "r", encoding="utf-8") as f:
    non_expert_results = json.load(f)

with open("expert_results.json", "r", encoding="utf-8") as f:
    expert_results = json.load(f)

In [12]:
# Calcular métricas para cada conjunto
non_expert_results = calculate_metrics(non_expert_results)
expert_results = calculate_metrics(expert_results)

In [13]:
# Imprimir resultados para verificación
print("Resultados para preguntas no expertas con métricas:")
for res in non_expert_results:
    print(res)
    
print("\nResultados para preguntas expertas con métricas:")
for res in expert_results:
    print(res)

Resultados para preguntas no expertas con métricas:
{'question': '¿Qué son los riesgos ESG?', 'expected_answer': 'Son aquellos riesgos relacionados con factores ambientales, sociales y de gobernanza que pueden afectar la estabilidad y funcionamiento de las instituciones financieras, incluyendo riesgos derivados de la transición hacia economías sostenibles y riesgos físicos (como eventos climáticos extremos).', 'generated_answer': 'Los riesgos ESG son:\n\nRiesgos ambientales, sociales y de gobernanza.', 'token_error_rate': 0.8421052631578947, 'bleu_score': 0.01689071874072545, 'rouge_l': 0.29268292314098754}
{'question': '¿Cuál es el objetivo principal de estas directrices ESG?', 'expected_answer': 'Establecer estándares mínimos y metodologías de referencia para identificar, medir, gestionar y monitorizar los riesgos ESG en las instituciones.', 'generated_answer': 'El objetivo principal de estas directrices ESG es:\n\nGestionar riesgos ESG', 'token_error_rate': 0.7894736842105263, 'bleu

In [14]:
# Guardar los resultados enriquecidos en nuevos archivos JSON
with open("non_expert_results_with_metrics.json", "w", encoding="utf-8") as f:
    json.dump(non_expert_results, f, ensure_ascii=False, indent=4)

with open("expert_results_with_metrics.json", "w", encoding="utf-8") as f:
    json.dump(expert_results, f, ensure_ascii=False, indent=4)
